<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RE24" data-toc-modified-id="RE24-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RE24</a></span><ul class="toc-item"><li><span><a href="#Load-Play-by-Play-Data-from-Retrosheet" data-toc-modified-id="Load-Play-by-Play-Data-from-Retrosheet-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Play-by-Play Data from Retrosheet</a></span></li><li><span><a href="#Load-Run-Expectancy-Matrix" data-toc-modified-id="Load-Run-Expectancy-Matrix-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load Run Expectancy Matrix</a></span></li><li><span><a href="#Run-Expectancies" data-toc-modified-id="Run-Expectancies-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Run Expectancies</a></span></li><li><span><a href="#Compute-RE24" data-toc-modified-id="Compute-RE24-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Compute RE24</a></span><ul class="toc-item"><li><span><a href="#Example" data-toc-modified-id="Example-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Example</a></span></li><li><span><a href="#Grand-Slams" data-toc-modified-id="Grand-Slams-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Grand Slams</a></span></li><li><span><a href="#Ichiro-Going-1st-to-3rd" data-toc-modified-id="Ichiro-Going-1st-to-3rd-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Ichiro Going 1st to 3rd</a></span></li></ul></li><li><span><a href="#Batter-RE24" data-toc-modified-id="Batter-RE24-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Batter RE24</a></span></li><li><span><a href="#On-Your-Own:-Run-Potential-and-RE24-for-Batters" data-toc-modified-id="On-Your-Own:-Run-Potential-and-RE24-for-Batters-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>On Your Own: Run Potential and RE24 for Batters</a></span><ul class="toc-item"><li><span><a href="#Run-Potential-and-RE24-in-the-Batting-Order" data-toc-modified-id="Run-Potential-and-RE24-in-the-Batting-Order-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Run Potential and RE24 in the Batting Order</a></span></li></ul></li></ul></li></ul></div>

# Demo - RE24

In this demo, we use run expectancy to create a valuation of hitting events that occur known as _RE24_.

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
from datascience import Table
from datascience.util import table_apply

import numpy as np

# custom functions that will help do some simple tasks
from datascience_utils import *
from datascience_stats import *
from datascience_topic import fast_run_expectancy, most_common_lineup_position

## RE24

The RE24 run production values for each plate appearance is computed as
$$
    \mathit{RE24} = \text{Run Expectancy}_{\text{Next}} - \text{Run Expectancy}_{\text{Current}} + \text{Runs Scored}
$$

We will use the retrosheet dataset we uses for the run expectancy demo and we will use the Run Expectancy Matrix from that demo as well.

### Load Play-by-Play Data from Retrosheet

Raw Retrosheet data (http://www.retrosheet.org/) contains play-by-play event logs, representing very raw information about the events in a baseball game. Lucky for us, the software program Chadwick (found here:http://chadwick.sourceforge.net/doc/index.html) was created to handle a lot of the messy work to compiled the data into a useable form.  Chadwick converts the raw logs into CSV which is what we use here.  Also, Chadwick computes some pretty important quantities that we make use of.

Note: This notebook uses data from 2001.  We could have used more recent data but Barry Bonds is a baseball god so part of this notebook is an excuse to revel in his statistical absurdity.

For computing the Run Expectancy Matrix as well as other analsis, we only need a few columns.  The relevant columns are:
+ EVENT_ID - An ID for the event in the dataset
+ INN_CT - Inning number
+ EVENT_CD - A code for what happened in the event
+ OUTS_CT - Number of outs
+ BAT_LINEUP_ID - Place in the batting order.  1 through 9.
+ BAT_EVENT_FL - A T/F flag as to whether the play-by-play event corresponds to a plate appearance (T) or some other type of event (F).
+ START_BASES_CD - An integer code representing the state of the runners, eg. runner on 2nd
+ END_BASES_CD - An integer code representing the state of the runners AFTER the event ends.
+ EVENT_OUTS_CT - Number of outs recorded on this event
+ EVENT_RUNS_CT - Number of runs scored on this event
+ FATE_RUNS_CT - Number of runs scored AFTER this event

We also make some modifications to the table.  See the demo on Run Expectancy for more information on these modifications.

In [ ]:
cols = ['EVENT_ID', 'INN_CT', 'EVENT_CD', 'OUTS_CT', 'BAT_ID', 'BAT_LINEUP_ID',
        'BAT_EVENT_FL', 'START_BASES_CD', 'END_BASES_CD', 'EVENT_OUTS_CT',
        'EVENT_RUNS_CT', 'FATE_RUNS_CT']
retro = Table.read_table('retrosheet_events-2001.csv.gz', sep=',', usecols=cols)

new_cols = ['ID', 'Inning', 'Event_Type', 'Outs', 'Batter_ID', 'Lineup_Order',
            'PA_Flag', 'Start_Bases', 'End_Bases', 'Event_Outs', 'Event_Runs',
            'Future_Runs']
retro.relabel(cols, new_cols)

bat_mask = (retro['PA_Flag'] == "T")
retro = retro.where(bat_mask).copy()

inning_mask = (retro['Inning'] < 9)
retro = retro.where(inning_mask).copy()

base_runner_codes = {
    0: "None on",  # No one on
    1: "1st",  # runner on 1st
    2: "2nd",  # runner on 2nd
    3: "1st and 2nd",  # runners on 1st & 2nd
    4: "3rd",  # runner on 3rd
    5: "1st and 3rd",  # runners on 1st & 3rd
    6: "2nd and 3rd",  # runners on 2nd & 3rd
    7: "Bases Loaded"  # bases loaded
}
# Replace the numeric code with a string code
retro['Start_Bases'] = replace(retro, 'Start_Bases', base_runner_codes)
retro['End_Bases'] = replace(retro, 'End_Bases', base_runner_codes)

event_codes = {
    0: 'Unknown',
    1: 'None',
    2: 'Generic out',
    3: 'K',  # Strikeout
    4: 'SB',  # Stolen Base
    5: 'Defensive indifference',
    6: 'CS',  # Caught stealing
    7: 'Pickoff error',
    8: 'Pickoff',
    9: 'Wild pitch',
    10: 'Passed ball',
    11: 'Balk',
    12: 'Other advance/out advancing',
    13: 'Foul error',
    14: 'BB',  # Walk
    15: 'IBB',  # Intentional walk
    16: 'HBP',  # Hit by pitch
    17: 'Interference',
    18: 'RBOE',  # Reached base on error
    19: 'FC',  # Fielder's choice
    20: '1B',  # Single
    21: '2B',  # Double
    22: '3B',  # Triple
    23: 'HR',  # Home run
    24: 'Missing play',
}

# Replace numeric code with string
retro['Event_Type'] = replace(retro, 'Event_Type', event_codes)

retro.show(10)

### Load Run Expectancy Matrix

We computed the Run Expectancy Matrix in the run expectancy demo so we load it and use it here.

In [ ]:
re = Table.read_table('re_2001.csv', sep=',')

### Run Expectancies

For each plate appearance, we need to compute the run expectancy that will serve as the backbone for computing the RE24 values.  To do this, we iterate through each PA, get its values for `Outs` and `Start_Bases` and extract the value from the Run Expectancy Matrix we computed earlier.  We do the same for the future state using `End_Bases` and `Outs + Event_Outs`.

In [ ]:
# NOTE: THIS CELL IS SLOW.
# It's easier to understand what the calculation is with this block of code but
# after that, you might as well run the next cell.

# # Iterate through PAs computing run expectancy
# run_exp = np.array([
#     re.where('Outs', outs).\
#         where('Start_Bases', base)['RE'].\
#         item() if outs < 3 else 0
#     for outs, base in zip(retro['Outs'], retro['Start_Bases'])
# ])
# retro['Run_Expectancy'] = run_exp

# # Iterate through PAs computing run expectancy for next PA
# next_outs = retro['Outs'] + retro['Event_Outs']
# next_run_exp = np.array([
#     re.where('Outs', outs).\
#         where('Start_Bases', base)['RE'].\
#         item() if outs < 3 else 0
#     for outs, base in zip(next_outs, retro['End_Bases'])
# ])
# retro['Run_Expectancy_Next'] = next_run_exp

In [ ]:
# This only takes about a second or two
retro = fast_run_expectancy(retro, re)

In [ ]:
view_cols = ['Batter_ID', 'Lineup_Order', 'Inning', 'Outs', 'Event_Type',
             'Start_Bases', 'End_Bases', 'Event_Runs', 'Run_Expectancy',
             'Run_Expectancy_Next']
retro.select(*[view_cols]).show(5)

### Compute RE24
Now its easy to compute the RE24 values using the fields we have computed.

In [ ]:
retro['RE24'] = retro['Run_Expectancy_Next'] - \
    retro['Run_Expectancy'] + retro['Event_Runs']

#### Example

Here are the first 10 rows from the Retrosheet table showing the first inning plus a bit of the second inning of the first game between the Angels and the Rangers.

In [ ]:
retro.select(*(view_cols + ['RE24'])).show(10)

#### Grand Slams

Below are three grand slams hit by Carl Everett, Barry Bonds, and Doug Mirabelli with different out situations.

First, notice how the high run expectancy at the beginning of the at-bat leads to a reduced or "discounted" RE24 value that does not equal 4.

Second, notice how the circumstances of the outs lead to receiving different values for the grand slam.

In [ ]:
retro.select(*(view_cols + ['RE24'])).take(23080)

In [ ]:
retro.select(*(view_cols + ['RE24'])).take(9090)

In [ ]:
retro.select(*(view_cols + ['RE24'])).take(4097)

#### Ichiro Going 1st to 3rd
A good baserunner like Ichiro (led the league in 2001 with 56 SB) that can get from 1st to 3rd on a hit easier can make the next hitter look better.

The following two pairs of events have Ichiro reaching 1st with no outs.  Compare the RE24 values of these two outcomes for the subsequent batters.  The events are identical except for Ichiro going to 3rd instead of 2nd on the single.  While the hitters are different (Mike Cameron vs Mark Mclemore) and we know little about the nature of the single that was hit, this shows a variation in values of RE24 for seemingly equal events.

In [ ]:
retro.select(*(view_cols + ['RE24'])).take[[112064, 112065]]

In [ ]:
retro.select(*(view_cols + ['RE24'])).take[[26273, 26274]]

_Question_
1. The underlying assumption of this approach is that the hitter receives all credit for everything on the basepaths.  Alternatively, we could ascribe all baserunning credit to the baserunner.  How might we adjust the RE24 computation to give Ichiro full credit for moving from second to third?  Does your computation apply more broadly to other situations?  What sort of information would we need?

### Batter RE24

We can group on `Batter_ID` and compute the Total RE24 for each batter.  We do this by just summing.

We don't have much context for how this statistic should behave since we only have one year and only view the top 10.  This can be done by diving into the statistic more on Baseball Reference or FanGraphs.  That said, it should be noted that (depending on how the statistic is ultimately computed and adjusted for various considerations), Barry Bonds' 2001, 2002, and 2004 seasons were absolutely obscene when it comes to offense and run production by any and all metrics.

In [ ]:
batter_data = retro.select('Batter_ID', 'Run_Expectancy', 'RE24').\
    group('Batter_ID', collect=sum)
batter_data.relabel(
    ['Run_Expectancy sum', 'RE24 sum'], ['Run_Expectancy', 'RE24'])
batter_data.sort('RE24', descending=True).show(10)

_Questions_
1. RE24 credits the batter with the run scored.  What is the assumption underlying this?  When would you want to give credit (or partial credit) to the baserunner?
2 What do you make of two Rockies players appearing on that list in 2001? (heltt001 is Todd Helton and walkl001 is Larry Walker)  What would be your first step in trying to adjust to reflect their favorable park?

### On Your Own: Run Potential and RE24 for Batters

Total RE24 for batters is a pretty cool stat given how we have accounted for run expectancy to try to better value events, especially ones that do not score runs.

But one thing we should wonder is how much run expectancy can vary in the batting order and how this relates to RE24.  Batting order and opportunity is a notorious problem for using RBI as a viable statistic and the analysis below corroborates that.

To do:
The function `most_common_lineup_order` takes in a Retrosheet dataset and finds the most common lineup position for each batter.  Using the derived most common lineup position for a batter, augment the `batter_data` in two ways:
1. Compute the total number of PAs for each batter.
2. Build a column containing two categories, "Qualified" and "Not Qualified", where $\mathit{PA} \geq 400$ means a batter is "Qualified".
3. Join `lineup_pos` and `batter_data` on `Batter_ID`.
4. Display Barry Bonds' results by selecting on his Batter ID `'bondb001'`.

In [ ]:
# Helper function to do fast groupby and find the most common 
# lineup position for each batter
lineup_pos = most_common_lineup_position(retro)

To do:
Generate a scatter plot on RE versus RE24 and color by whether the hitter is Qualified or not (ie. has over 400 PA or not).

_Questions_
1. What is the relationship between total number of PAs and Total Run Expectancy?
2. For qualified hitters, what is the relationship between Total RE and Total RE24?  Why do you think the relationship is the way it appears?  What does it say about run expectancy, the quality of a batter, and the batter's run production?

#### Run Potential and RE24 in the Batting Order
Let's use the Retrosheet data to compute total RE and RE24 by place in the batting order.  Here we can try to see the direct effects of how the batting order affects run production.

To do: From the Retrosheet table, derive a new table `batting_order_totals`.  This table should contain three columns:
 + Lineup_Order
 + Total Run Expectancy for each position in the lineup
 + Total RE24 for each position in the lineup

##### An Odd Finding...

We want to know about the relationships between run expectancy and RE24 depending on the lineup position?

To do:
1. Plot RE and RE24 against the lineup position.

_Question_
1. Which lineup position has the highest total run potential?  Why?  Which did you expect to have the highest total run potential?
2. Which lineup position has the highest total run production?  Does this seem right?

##### ...Easily Explained

To do:
1. Augment `batting_order_totals` with total PA for each lineup position.  Plot the lineup position against PA.

_Question_
1. What obvious result do you see?  Why is this so obvious?  What does it mean about the previous result?

##### Normalize by Number of PAs

To do:
1. Augment `batting_order_totals` with two columns: `'RE per PA'` and `'RE24 per PA'`.  These two quantities are self-explanatory.
2. Plots `'RE per PA'` and `'RE24 per PA'` against lineup position.

_Question_
+ Does lineup position, run expectancy, and run production make sense now?

##### Boxplots by Lineup Position

Plotting the average is not ideal so with a bit of extract work and a helper function that leverages the boxplot functionality of Pandas, we can view a boxplot of the run potential and production per plate appearance _for each player_ grouped by lineup order.  

You can see Barry Bonds' ridiculous performance in the second plot.  He's the \#3 hitter with RE24 per PA of 0.175.  If you dig into the data more you find that Bonds did not have a particularly high run potential: 0.5 runs per PA.

In [ ]:
if 'Lineup_Order' in batter_data.labels:
    re_per_pa = batter_data['Run_Expectancy'] / batter_data['PA']
    re24_per_pa = batter_data['RE24'] / batter_data['PA']
    batter_data = batter_data.with_columns(
        'RE per PA', re_per_pa,
        'RE24 per PA', re24_per_pa
    )
    qualified_batters = batter_data.where('qual', 'Qualified')
    # boxplot_pd is a helper function to use Pandas-style boxplot visualizations
    boxplots(qualified_batters, column='RE per PA', by='Lineup_Order')
    boxplots(qualified_batters, column='RE24 per PA', by='Lineup_Order')

In [ ]:
if 'Lineup_Order' in batter_data.labels:
    qualified_batters.sort('RE24 per PA', descending=True).show()

_Question_
+ What's a plausible explanation for why the _most productive_ hitter appears to bat third but the fourth and fifth place hitters bat in the most run rich environments?   

+ We figured out that run potential depends on the batting order.  We saw a relationship that showed total run production is associated with total run potential.  We have two plausible reasons for this: (1) higher run potential inherently drives higher run production, and (2), high run producers are put in high run potential situations.  But have we actually been able to figure out whether run production is measuring an ability rather than being a product of environment?